In [ ]:
%matplotlib inline
import os
import mujoco
import numpy as np
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from typing import Dict, Union
import matplotlib.pyplot as plt
import cv2


DEFAULT_CAMERA_CONFIG = {
    "trackbodyid": -1,
    "distance": 4.0,
}

class MyRobotEnv(MujocoEnv):
    def __init__(
        self, 
        xml_path = "/workspace/ros2_ws/src/airhockey2025/ka/assets/main.xml",
        frame_skip: int = 5,
        default_camera_config: Dict[str, Union[float, int]] = DEFAULT_CAMERA_CONFIG,
        **kwargs,
    ):
        # 一度モデルを読み込んで観測次元を取得
        model = mujoco.MjModel.from_xml_path(xml_path)
        obs_dim = model.nq + model.nv
        observation_space = Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)

        # 行動空間：joint velocities [-1, 1] 正規化
        # 現状はmodelのアクチュエータが認識していないので、model.nuがなくなっている
        # そのため, model.ctrlもないのでstepができない状態
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=model.actuator_actnum.shape, dtype=np.float32
        )
        
        # 親クラスの初期化
        super().__init__(
            model_path=xml_path,
            frame_skip=frame_skip,
            observation_space=observation_space,
            render_mode="rgb_array",
            default_camera_config=default_camera_config,
            **kwargs,
        )
        
        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
                "rgbd_tuple",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

    def step(self, action):
        # 正規化されたactionをスケーリング
        scaled_action = action * self.model.actuator_ctrlrange[:, 1]
        self.do_simulation(scaled_action, self.frame_skip)

        obs = self._get_obs()
        reward = self._compute_reward(obs, action)
        done = False
        info = {}

        return obs, reward, done, info

    def _get_obs(self):
        return np.concatenate([self.data.qpos.flat, self.data.qvel.flat])

    def _compute_reward(self, obs, action):
        # エンドエフェクタの位置を使用した報酬例
        # end_effector_pos = self.data.site_xpos[self.model.site_name2id("ee_site")]
        # goal = np.array([0.5, 0.0, 0.2])
        # dist = np.linalg.norm(end_effector_pos - goal)
        # return -dist
        return 0

    def reset_model(self):
        # ランダム初期化
        qpos = self.init_qpos + np.random.uniform(-0.01, 0.01, size=self.model.nq)
        qvel = self.init_qvel + np.random.uniform(-0.01, 0.01, size=self.model.nv)
        self.set_state(qpos, qvel)
        return self._get_obs()

In [43]:
def render(images):
    for img in images:
        # 画像をuint8のBGRに変換（OpenCV用）
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        cv2.imshow('Video', img_bgr)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [ ]:
env = MyRobotEnv()
env.reset()
images = []
skip_frame = 5

for i in range(1000):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    if i % skip_frame == 0:
        images.append(env.render())

render(images)